# 用 SkillCorner `matching=statsbomb` 做 StatsBomb ↔ SkillCorner 数据对齐

目标：从 StatsBomb 和 SkillCorner 爬取数据后，通过 **ID 匹配** 或 **模糊匹配** 将两方数据对应起来。

发现：SkillCorner 提供 `get_teams(params={'matching': 'statsbomb'})`，返回 SkillCorner 中与 StatsBomb 有关联的球队，部分球队带 `statsbomb_id` 字段，可直接用于 ID 精确匹配。

In [5]:
import sys
import os

_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "API" else os.getcwd()
sys.path.insert(0, _root)

from skillcorner.client import SkillcornerClient
from config import SKILLCORNER_USERNAME, SKILLCORNER_PASSWORD

client = SkillcornerClient(username=SKILLCORNER_USERNAME, password=SKILLCORNER_PASSWORD)
print("SkillCorner 客户端已连接")

SkillCorner 客户端已连接


## 1. 调用 `get_teams(params={'matching': 'statsbomb'})`

返回 SkillCorner 中“与 StatsBomb 数据有匹配”的球队。核心字段：
- `id`：SkillCorner 球队 ID
- `name`：球队名
- `statsbomb_id`：若 SkillCorner 已建立映射，则为 StatsBomb 的球队 ID；否则为 `None`

In [6]:
teams = client.get_teams(params={'matching': 'statsbomb'})
print(f"总数量: {len(teams)} 支球队")

# 统计有 statsbomb_id vs 无 statsbomb_id
with_sb_id = [t for t in teams if t.get('statsbomb_id')]
without_sb_id = [t for t in teams if not t.get('statsbomb_id')]
print(f"\n有 statsbomb_id（可直接 ID 匹配）: {len(with_sb_id)}")
print(f"无 statsbomb_id（需模糊匹配）: {len(without_sb_id)}")

总数量: 4929 支球队

有 statsbomb_id（可直接 ID 匹配）: 1840
无 statsbomb_id（需模糊匹配）: 3089


In [7]:
# 有 statsbomb_id 的示例
print("有 statsbomb_id 的示例:")
for t in with_sb_id[:5]:
    print(f"  SC id={t['id']}, name={t['name']}, statsbomb_id={t['statsbomb_id']}")

# 无 statsbomb_id 的示例
print("\n无 statsbomb_id 的示例:")
for t in without_sb_id[:5]:
    print(f"  SC id={t['id']}, name={t['name']}, statsbomb_id={t.get('statsbomb_id')}")

有 statsbomb_id 的示例:
  SC id=2690, name=FK Proleter Novi Sad, statsbomb_id=1275
  SC id=2503, name=Liebherr Grazer AK, statsbomb_id=19672
  SC id=2492, name=Criciuma EC, statsbomb_id=5692
  SC id=2514, name=Modena FC, statsbomb_id=11011
  SC id=2520, name=NK Inter Zaprešić, statsbomb_id=1339

无 statsbomb_id 的示例:
  SC id=2486, name=CR Flamengo U20, statsbomb_id=None
  SC id=2495, name=Queen of the South FC, statsbomb_id=None
  SC id=2489, name=Volta Redonda FC, statsbomb_id=None
  SC id=4044, name=Hong Kong FC, statsbomb_id=None
  SC id=1930, name=Moldova, Women, statsbomb_id=None


## 2. 构建 SkillCorner ↔ StatsBomb 球队映射

对有 `statsbomb_id` 的球队，可直接建立 `(skillcorner_team_id, statsbomb_team_id)` 映射，用于后续 ingest 时的精确匹配，替代按名称的模糊匹配。

In [8]:
# 映射: skillcorner_team_id -> statsbomb_team_id
sc_to_sb_team = {t['id']: int(t['statsbomb_id']) for t in teams if t.get('statsbomb_id')}

# 反向: statsbomb_team_id -> skillcorner_team_id
sb_to_sc_team = {int(t['statsbomb_id']): t['id'] for t in teams if t.get('statsbomb_id')}

print(f"可直接 ID 匹配的球队对数: {len(sc_to_sb_team)}")
print("\n映射示例 (SC -> SB):")
for i, (sc_id, sb_id) in enumerate(list(sc_to_sb_team.items())[:5]):
    name = next(t['name'] for t in teams if t['id'] == sc_id)
    print(f"  {name}: SC {sc_id} -> SB {sb_id}")

可直接 ID 匹配的球队对数: 1840

映射示例 (SC -> SB):
  FK Proleter Novi Sad: SC 2690 -> SB 1275
  Liebherr Grazer AK: SC 2503 -> SB 19672
  Criciuma EC: SC 2492 -> SB 5692
  Modena FC: SC 2514 -> SB 11011
  NK Inter Zaprešić: SC 2520 -> SB 1339


## 3. 与当前 ingest 方式的区别

| 方式 | 当前 `skillcorner_ingestion.py` | 使用 `matching=statsbomb` |
|------|--------------------------------|---------------------------|
| 数据源 | `get_teams(competition_edition=X)` 只拿某一联赛赛季 | `get_teams(matching='statsbomb')` 拿所有有 StatsBomb 关联的球队 |
| 匹配策略 | 按球队名称 LIKE 模糊匹配 | 有 `statsbomb_id` 时用 ID 精确匹配，无则仍需名称模糊匹配 |
| 用途 | 针对 Ligue 1 等特定联赛 ingest | 建立全局 SC↔SB 映射，适合多联赛或需要精确 ID 的场景 |

In [9]:
# 可选：筛选 Ligue 1 相关球队，看有多少能通过 statsbomb_id 匹配
# 需结合 competition_edition 获取该赛季球队列表，再与本列表求交
ligue1_keywords = ['fc metz', 'psg', 'lyon', 'marseille', 'lens', 'monaco', 'nice', 'lille']
ligue1_like = [t for t in teams if any(kw in (t.get('name') or '').lower() for kw in ligue1_keywords)]
ligue1_with_id = [t for t in ligue1_like if t.get('statsbomb_id')]
print("Ligue 1 相关球队中有 statsbomb_id 的:")
for t in ligue1_with_id:
    print(f"  {t['name']}: statsbomb_id={t['statsbomb_id']}")

Ligue 1 相关球队中有 statsbomb_id 的:
  Lille OSC Métropole, Women: statsbomb_id=19108
  LD Alajuelense: statsbomb_id=4939
  Lillestrom SK: statsbomb_id=369
  Nublense: statsbomb_id=5102
  Olympique Lyonnais, Women: statsbomb_id=922
  RC Lens: statsbomb_id=168
  OGC Nice: statsbomb_id=136
  LOSC Lille: statsbomb_id=143
  FC Metz: statsbomb_id=141
  AS Monaco: statsbomb_id=129
  Olympique de Marseille: statsbomb_id=147
  Olympique Lyonnais: statsbomb_id=137
  AS Lyon-Duchere: statsbomb_id=2383
